<a href="https://colab.research.google.com/github/cove9988/catboost-icarus/blob/main/build_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install catboost
!pip install pandas-profiling 
!pip install shap


     |████████████████████████████████| 67.4 MB 32 kB/s 
     |████████████████████████████████| 356 kB 5.4 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491658 sha256=bc2b19db701416a5875fd9cec3660d2a1c30783e6c80d5d5f6fea08adde99f8a
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [3]:
import numpy as np
import pandas as pd
# import statsmodels.api as m
from catboost import CatBoostRegressor, Pool
import shap
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import  pandas_profiling as pp

In [6]:
#!pip install dash
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import drive
from plotly import graph_objects as go
drive.mount('/content/gdrive')
np.set_printoptions(precision=4)
gdrive_path='gdrive/MyDrive/Colab Notebooks'

Mounted at /content/gdrive


In [7]:
raw = pd.read_csv(f'{gdrive_path}/raw.csv')

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
raw = pd.read_csv(io.StringIO(uploaded['raw.csv'].decode('utf-8')))


In [8]:
raw.set_index('time')
raw.index = pd.to_datetime(raw.index,unit='s')

In [9]:
def ma_shift(dataset, ma_period:int, point, look_back:int = 20, head_cnt = 1000, inital='') :
    rate_frame = dataset.copy(deep=True)
    rate_ma = rate_frame['close'].rolling(ma_period).mean()
    rate_diff = (rate_frame['close'] - rate_ma)/point
    for i in range(look_back):
        rate_frame[f'{inital}_{str(i)}'] = rate_diff.shift(i)
    rate_frame = rate_frame.dropna()
    return rate_frame.head(n=head_cnt)

def stochastic(ticker,stoch_period=14,sm_average=3):
  ticker[f'{stoch_period}-high'] = ticker['high'].rolling(stoch_period).max()
  ticker[f'{stoch_period}-low'] = ticker['low'].rolling(stoch_period).min()
  ticker['%K'] = (ticker['close'] - ticker[f'{stoch_period}-low'])*100/(ticker[f'{stoch_period}-high'] - ticker[f'{stoch_period}-low'])
  ticker['%D'] = ticker['%K'].rolling(sm_average).mean()
  return ticker

def add_labels(dataset,point, expected_point,max_step):
    labels =[]
    length = dataset.shape[0]-max_step
    # print(f'total data length: {length}')
    for i in range(length):
        print(f'line ------------ {i}')
        max_ll, max_hh =0,0
        for r in  range(max_step):
            
            # print(dataset['close'][i] , dataset['low'][i + r])
            try:           
              ll =(dataset['close'][i] - dataset['low'][i + r])/point
              hh =(dataset['high'][i + r] - dataset['close'][i])/point
            except:
              print('error here')
            # print(f'search ------------ {r} ll:{ll} hh:{hh}')
            if ll > expected_point and hh > expected_point:
                print(f'{i} -{r} reach both sides')
                labels.append(0)        
                break 
            elif hh > expected_point:
                print(f'{i} -{r} reach HH sides {roundUpToMultiple(hh)}' )
                labels.append(roundUpToMultiple(hh))         
                break
            elif ll > expected_point:
                print(f'{i} -{r} reach LL sides {roundUpToMultiple(-ll)}')
                labels.append(roundUpToMultiple(-ll))         
                break
            else:
                max_ll = max_ll if ll < max_ll else ll
                max_hh = max_hh if hh < max_hh else hh
            
            # reach the range border
            if r == max_step -1: 
                if max_ll > max_hh:
                    print(f'{i} -{r} ll: {roundUpToMultiple(-max_ll)}')
                    labels.append(roundUpToMultiple(-max_ll))
                else:
                    print(f'{i} -{r} hh: {roundUpToMultiple(max_hh)}')
                    labels.append(roundUpToMultiple(max_hh))
                continue
                    
    dataset = dataset.iloc[:len(labels)].copy()
    dataset['labels'] = labels
    dataset = dataset.dropna()
    return dataset

def roundUpToMultiple(number, multiple=5):
    if number >0:
      num = number - (multiple +1)
    else:
      num = number + (multiple-1)
    return int(num - (num % multiple))

In [10]:

LOOK_BACK = 24
MA_PERIOD_SHORT = 15
MA_PERIOD_LONG = 50
MARKUP = 0.0000
POINT = 0.00001
ma = ma_shift(raw,MA_PERIOD_SHORT,POINT, LOOK_BACK,1000000, 'MA_15')
ma2 = ma_shift(ma,MA_PERIOD_LONG,POINT, LOOK_BACK,1000000, 'MA_50')
stoch = stochastic(ma2)
print(stoch)
mdataset = add_labels(stoch,POINT, 300,24)


Streaming output truncated to the last 5000 lines.
line ------------ 220791
220791 -23 ll: -65
line ------------ 220792
220792 -23 ll: -80
line ------------ 220793
220793 -23 ll: -65
line ------------ 220794
220794 -23 hh: 60
line ------------ 220795
220795 -23 hh: 75
line ------------ 220796
220796 -23 hh: 65
line ------------ 220797
220797 -23 hh: 45
line ------------ 220798
220798 -23 ll: -40
line ------------ 220799
220799 -23 ll: -60
line ------------ 220800
220800 -23 ll: -50
line ------------ 220801
220801 -23 ll: -40
line ------------ 220802
220802 -23 ll: -40
line ------------ 220803
220803 -23 ll: -50
line ------------ 220804
220804 -23 ll: -55
line ------------ 220805
220805 -23 ll: -55
line ------------ 220806
220806 -23 ll: -55
line ------------ 220807
220807 -23 ll: -40
line ------------ 220808
220808 -23 ll: -50
line ------------ 220809
220809 -23 ll: -65
line ------------ 220810
220810 -23 ll: -55
line ------------ 220811
220811 -23 ll: -50
line ------------ 220812
2208

In [11]:
outcome_name = 'labels'
not_feature_list = ['labels','time']
features_for_model = [f for f in list(mdataset) if f not in not_feature_list]
print(features_for_model)
x_train, x_test, y_train, y_test = train_test_split(mdataset[features_for_model],mdataset[outcome_name],test_size=0.3, random_state=1)

['open', 'high', 'low', 'close', 'MA_15_0', 'MA_15_1', 'MA_15_2', 'MA_15_3', 'MA_15_4', 'MA_15_5', 'MA_15_6', 'MA_15_7', 'MA_15_8', 'MA_15_9', 'MA_15_10', 'MA_15_11', 'MA_15_12', 'MA_15_13', 'MA_15_14', 'MA_15_15', 'MA_15_16', 'MA_15_17', 'MA_15_18', 'MA_15_19', 'MA_15_20', 'MA_15_21', 'MA_15_22', 'MA_15_23', 'MA_50_0', 'MA_50_1', 'MA_50_2', 'MA_50_3', 'MA_50_4', 'MA_50_5', 'MA_50_6', 'MA_50_7', 'MA_50_8', 'MA_50_9', 'MA_50_10', 'MA_50_11', 'MA_50_12', 'MA_50_13', 'MA_50_14', 'MA_50_15', 'MA_50_16', 'MA_50_17', 'MA_50_18', 'MA_50_19', 'MA_50_20', 'MA_50_21', 'MA_50_22', 'MA_50_23', '14-high', '14-low', '%K', '%D']


In [ ]:
params ={
    'iterations':10000,
    'learning_rate':0.001,
    'depth':6,
    'loss_function':'RMSE',
    'random_seed':0,
    'od_type':'Iter',
    'od_wait':20,
    'verbose':200,
    'use_best_model':True,
    'task_type':'GPU',
}
model_regressor = CatBoostRegressor(**params)
model_regressor.fit(x_train,y_train,
                    eval_set=(x_test,y_test),
                    use_best_model=True,
                    plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 164.1950930	test: 164.2303278	best: 164.2303278 (0)	total: 14.4ms	remaining: 2m 24s
200:	learn: 164.0854542	test: 164.1526761	best: 164.1526761 (200)	total: 2.22s	remaining: 1m 48s
400:	learn: 163.9913540	test: 164.0898831	best: 164.0898831 (400)	total: 4.35s	remaining: 1m 44s
600:	learn: 163.9073832	test: 164.0359841	best: 164.0359841 (600)	total: 6.48s	remaining: 1m 41s
800:	learn: 163.8311578	test: 163.9892922	best: 163.9892922 (800)	total: 8.6s	remaining: 1m 38s
1000:	learn: 163.7587078	test: 163.9456350	best: 163.9456350 (1000)	total: 10.7s	remaining: 1m 36s
1200:	learn: 163.6899832	test: 163.9050150	best: 163.9050150 (1200)	total: 12.8s	remaining: 1m 33s
1400:	learn: 163.6241878	test: 163.8665773	best: 163.8665773 (1400)	total: 15s	remaining: 1m 31s
1600:	learn: 163.5606693	test: 163.8308484	best: 163.8308484 (1600)	total: 17.1s	remaining: 1m 29s
1800:	learn: 163.4987140	test: 163.7962666	best: 163.7962666 (1800)	total: 19.1s	remaining: 1m 27s
2000:	learn: 163.4394060	t

In [ ]:
feat_import = [t for t in zip(features_for_model,model_regressor.get_feature_importance())]
feat_import_df = pd.DataFrame(feat_import, columns=['Feature','VarImp'])
feat_import_df = feat_import_df.sort_values('VarImp', ascending=False)
feat_import_df.head(100)

In [ ]:
m_categories= np.where(mdataset[features_for_model].dtypes != np.float)[0]
print(f'categories with not float : {m_categories}')

In [ ]:
shap_values = model_regressor.get_feature_importance(Pool(x_test, label=y_test, cat_features=m_categories) , type="ShapValues")
 
expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]

shap.initjs()
shap.force_plot(expected_value, shap_values[0,:], x_test.iloc[0,:])

In [ ]:
shap.summary_plot(shap_values, x_test)